In [1]:
import findspark

In [2]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName('Basics').getOrCreate()

23/07/06 18:48:14 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/06 18:48:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 18:48:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.ml.regression import LinearRegression

In [10]:
data=spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [11]:
data.printSchema()


root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [15]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [16]:
output=assembler.transform(data)

In [17]:
final_data=output.select('features','Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                352|
|   mean| 497.78882747251095|
| stddev|  80.83808661437017|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                148|
|   mean| 502.94156661467235|
| stddev|  75.71188657546693|
|    min| 347.77692663187264|
|    max|  744.2218671047146|
+-------+-------------------+



In [22]:
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lr_model=lr.fit(train_data)

23/07/06 19:10:24 WARN Instrumentation: [3dd46dcf] regParam is zero, which might cause numerical instability and overfitting.
23/07/06 19:10:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/07/06 19:10:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [24]:
test_results=lr_model.evaluate(test_data)

In [25]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  7.335612203569099|
| -4.284619325917447|
|-18.102776974614926|
| 16.040565946152412|
|-2.6988383274789953|
|  3.556701139280051|
|  -4.93502112895186|
| -19.41584591238899|
| -5.112407803078156|
|-17.395764978294494|
| 12.604328801758697|
|-17.961207777040727|
| 0.3799656939992815|
|  4.705288820086139|
| 16.024006815017344|
| 1.3410736978260047|
| 2.1893058786396296|
| 19.876943864191162|
|-4.9077023297091955|
| -4.771439748849673|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.192082231262845

In [28]:
test_results.r2

0.9817550749939612

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data=test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.9716756438877...|
|[31.2681042107507...|
|[31.5702008293202...|
|[31.6098395733896...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.7656188210424...|
|[31.8164283341993...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9096268275227...|
|[31.9563005605233...|
|[32.0047530203648...|
|[32.0478009788678...|
|[32.0478146331398...|
|[32.0705462209254...|
|[32.1164001209281...|
|[32.1253868972878...|
|[32.1338624098483...|
|[32.1490605221059...|
+--------------------+
only showing top 20 rows



In [33]:
predictions=lr_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.9716756438877...| 487.3029975533236|
|[31.2681042107507...|427.75515249974137|
|[31.5702008293202...| 564.0482691160198|
|[31.6098395733896...|428.50498370495575|
|[31.7216523605090...|350.47576495935164|
|[31.7366356860502...|493.37674511625187|
|[31.7656188210424...|  501.489102764559|
|[31.8164283341993...| 520.5383374160454|
|[31.8745516945853...|397.39765204934565|
|[31.9048571310136...|491.34562240111063|
|[31.9096268275227...| 550.8417068714805|
|[31.9563005605233...| 565.0871395242395|
|[32.0047530203648...|463.36601542663016|
|[32.0478009788678...| 508.7452823660103|
|[32.0478146331398...|481.36555094382607|
|[32.0705462209254...| 531.4107138840104|
|[32.1164001209281...|461.29568952562636|
|[32.1253868972878...| 437.9707520807574|
|[32.1338624098483...| 448.3495623921715|
|[32.1490605221059...|397.76369566288054|
+--------------------+------------